In [ ]:
import cv2
import numpy as np

import ipywidgets as widgets
from IPython.display import display, clear_output
import time

from tiki.mini import TikiMini

In [ ]:
tiki = TikiMini()

In [ ]:
# Hyper Parameters
lower_green = np.array([67, 23, 101])
upper_green = np.array([80, 255, 168])

lower_line = np.array([67, 23, 101])
upper_line = np.array([80, 255, 168])

rho = 1
theta = np.pi / 180
threshold = 80
min_line_length = 50
max_line_gap = 10
angle_threshold = 30

In [3]:
class Wrapper:
    def __init__(self, save_name=None):
        self.cap = cv2.VideoCapture(
            "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=640, height=480, framerate=15/1, format=NV12 ! "
            "nvvidconv flip-method=2 ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
        )
    def __enter__(self):
        print('entered')
        return self.cap
    def __exit__(self,a,b,c):
        self.cap.release()
        print('released')

In [ ]:
def detect_lines(image, rho, theta, threshold, min_line_length, max_line_gap):
    crop_img = image[240:480,150:480].copy()
    gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
    edges = cv2.Canny(binary, 50, 150, apertureSize=3)
    lines = cv2.HoughLineWP(edges, rho, theta, threshold, minLineLength=min_line_length, maxLineGap=max_line_gap)

    if lines is None:
        return image, crop_img, [], None, 0

    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(crop_img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    image[240:480, 150:480] = crop_img
    
    return image, crop_img, lines

In [ ]:
def calculate_steering_angle(left_lines, right_lines):
    left_slope = np.mean([((line[0][3] - line[0][1]) / (line[0][2] - line[0][0])) for line in left_lines]) if left_lines else 0
    right_slope = np.mean([((line[0][3] - line[0][1]) / (line[0][2] - line[0][0])) for line in right_lines]) if right_lines else 0
    
    if left_slope == 0 and right_slope == 0:
        return 0
    
    if abs(left_slope) > abs(right_slope):
        steering_angle = np.arctan(left_slope)
    else:
        steering_angle = np.arctan(right_slope)
    
    steering_angle = np.degrees(steering_angle)
    
    # Normalize steering angle to be between -1 and 1
    max_angle = 45  # Adjust this value based on your specific requirements
    normalized_angle = steering_angle / max_angle
    
    return np.clip(normalized_angle, -1, 1)

In [ ]:
def adjust_wheel_speed(steering_angle, base_speed=20):
    left_speed = base_speed
    right_speed = base_speed
    
    if steering_angle > 0:  # Turn right
        left_speed += steering_angle * base_speed
        right_speed -= steering_angle * base_speed

    else:  # Turn left
        left_speed += abs(steering_angle) * base_speed
        right_speed -= abs(steering_angle) * base_speed
    
    return int(left_speed), int(right_speed)

In [ ]:
# 카메라에서 실시간 이미지 출력하는 함수
def main():
    video_widget = widgets.Image(format='jpeg')
    tiki.set_motor_mode(tiki.MOTOR_MODE_PID)
    with Wrapper() as cap:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            result, crop_result, lines= detect_lines(frame, rho, theta, threshold, min_line_length, max_line_gap)
            left_lines, right_lines = lines

            steering_angle = calculate_steering_angle(left_lines, right_lines)
            left_speed, right_speed = adjust_wheel_speed(steering_angle)
            tiki.set_motor_power(tiki.MOTOR_LEFT, left_speed)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, right_speed)

            left_encoder = tiki.get_encoder(tiki.MOTOR_LEFT)
            right_encoder = tiki.get_encoder(tiki.MOTOR_RIGHT)
            print(f"Left Encoder: {left_encoder}, Right Encoder: {right_encoder}")

            _, buffer = cv2.imencode('.jpg', result)
            video_widget.value = buffer.tobytes()
            
            clear_output(wait=True)
            display(video_widget)
            time.sleep(0.03)  # 프레임 간 간격

if __name__ == '__main__':
    main()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

released


KeyboardInterrupt: 